# HPO: obs_days=28

CQL Hyperparameter Optimization with:
- obs_days: 28
- grid_step_hours: 8
- inclusion: kdigo2

In [ ]:
# ============================================================
# CONFIGURATION - ADJUST THESE VALUES
# ============================================================
OBS_DAYS = 28
GRID_HOURS = 8
INCLUSION = 'kdigo2'
DATABASE = 'aumc'  # 'aumc' or 'mimic'

# Reward settings
MORTALITY_PENALTY = 'zero'  # Options: 'no', 'zero', 'penalty', 'both'
DISCOUNT = 1.0

# Algorithms to run
RUN_CQL = True
RUN_DDQN = True
RUN_BCQ = False
RUN_NFQ = False

In [ ]:
# 1. GPU Check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 2. Install dependencies
!pip install d3rlpy optuna pandas matplotlib seaborn pyyaml scikit-learn tqdm -q
print("Dependencies installed!")

In [ ]:
# 3. Clone repository
!git clone https://github.com/karel2324/Thesis.git
%cd Thesis/RRT_start/2_Total

In [ ]:
# 4. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 5. Apply configuration
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Dataset config
config['dataset']['obs_days'] = OBS_DAYS
config['dataset']['grid_step_hours'] = GRID_HOURS
config['dataset']['inclusion'] = INCLUSION

# Reward config
config['reward']['mortality_penalty'] = MORTALITY_PENALTY
config['reward']['discount'] = DISCOUNT

# Algorithm selection
config['hpo']['algorithms'] = {
    'cql': RUN_CQL,
    'ddqn': RUN_DDQN,
    'bcq': RUN_BCQ,
    'nfq': RUN_NFQ
}
config['hpo']['databases'] = {
    'aumc': DATABASE == 'aumc',
    'mimic': DATABASE == 'mimic'
}

with open('config.yaml', 'w') as f:
    yaml.dump(config, f)

# Print summary
db_name = 'AUMCdb' if DATABASE == 'aumc' else 'MIMIC'
print(f"Config: {db_name} obs{OBS_DAYS} grid{GRID_HOURS} {INCLUSION}")
print(f"  obs_days: {OBS_DAYS}")
print(f"  grid_step_hours: {GRID_HOURS}")
print(f"  inclusion: {INCLUSION}")
print(f"  mortality_penalty: {MORTALITY_PENALTY}")
print(f"  discount: {DISCOUNT}")
print(f"  algorithms: CQL={RUN_CQL}, DDQN={RUN_DDQN}, BCQ={RUN_BCQ}, NFQ={RUN_NFQ}")

In [ ]:
# 6. Verify data exists
db_prefix = 'AUMCdb' if DATABASE == 'aumc' else 'MIMIC'
data_path = f"/content/drive/MyDrive/Thesis/Data/derived_{db_prefix}_obs{OBS_DAYS}_grid{GRID_HOURS}_{INCLUSION}/"
!ls -la {data_path}

In [ ]:
# 7. Run HPO
!python _7_hpo.py

In [ ]:
# 8. Check results
db_prefix = 'AUMCdb' if DATABASE == 'aumc' else 'MIMIC'
results_path = f"/content/drive/MyDrive/Thesis/Data/derived_{db_prefix}_obs{OBS_DAYS}_grid{GRID_HOURS}_{INCLUSION}/Reward_*/HPO_results/"
!ls -la {results_path}